In [122]:
from __future__ import print_function
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path)

import numpy as np
import math
import copy
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist
import scipy.stats as ss
import time
import pickle
import matplotlib.pylab as plt
import matplotlib.patches as mpatches
from joblib import Parallel, delayed

In [123]:
def quantize_tensor(x, num_bits, min_val=None, max_val=None):
  
  if not min_val and not max_val: 
      min_val, max_val = x.min(), x.max()

  qmin = -2.**(num_bits-1)
  qmax = 2.**(num_bits-1) - 1.

  x = x - min_val          # Allineo tutto l'array in modo che parta da 0
  x /= (max_val - min_val) # Lo scalo tra 0 e 1    
  x *= (qmax - qmin)       # Lo scalo tra 0 e 16
  x -= qmax                # Lo sfaso tra -8 e 7
  
  q_x = x.astype(float).round().astype(int)
  
  return q_x

In [124]:
# Prepare Penguins dataset
penguins = pd.read_csv('../penguins_size.csv')
penguins = penguins.sample(frac=1, random_state=2)
penguins = penguins.dropna()

x_train, y_train = penguins.loc[:, ["island", "culmen_length_mm", "flipper_length_mm", "body_mass_g"]].values, penguins.iloc[:, :1].values
for i in range(len(y_train)):
  if y_train[i][0] == "Adelie":
    y_train[i][0] = 0
  elif y_train[i][0] == "Gentoo":
    y_train[i][0] = 1
  else:
    y_train[i][0] = 2

island, sex = {}, {}
countI, countS = 0, 0

for i in range(len(x_train)):
  # Island
  if x_train[i][0] in island:
    x_train[i][0] = island[x_train[i][0]]
  else:
    island[x_train[i][0]] = countI
    x_train[i][0] = countI
    countI += 1

x_train[:, 1] = quantize_tensor(x_train[:, 1], 4)
x_train[:, 2] = quantize_tensor(x_train[:, 2], 4)
x_train[:, 3] = quantize_tensor(x_train[:, 3], 4)

train, val = 150, 64
x_val, y_val = x_train[train:train+val], y_train[train:train+val]
x_test, y_test = x_train[train+val:], y_train[train+val:]
y_train = np_utils.to_categorical(y_train).astype(int)*8
x_train, y_train = x_train[:train], y_train[:train]

## Net Architecture

In [125]:
SHRT_MAX = 32767
SHRT_MIN = (-SHRT_MAX - 1 )

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [126]:
# DFA WEIGHTS
def randomDfaWeightUniform(mInDim, mOutDim):
  range = isqrt((12 * SHRT_MAX) / (mInDim + mOutDim))
  return np.random.randint(-range, range, (mInDim, mOutDim))

In [127]:
# DFA WEIGHTS
def randomDfaWeightNormal(mInDim, mOutDim):
  range = isqrt((12 * SHRT_MAX) / (mInDim + mOutDim))
  x = np.arange(-range, range)
  xU, xL = x + 0.5, x - 0.5
  prob = ss.norm.cdf(xU, scale = range//10) - ss.norm.cdf(xL, scale = range//10)
  prob = prob / prob.sum() # normalize the probabilities so their sum is 1
  nums = np.random.choice(x, size = (mInDim, mOutDim), p = prob)
  return nums.astype(int)

In [128]:
def pocketTanh(matIn, bits, inDims, outDims):
    yMax = 128
    yMin = -127
    joints = [128, 75, 32, -31, -74, -127]
    divisor = (1 << bits) * inDims
    slopesInv = [yMax, 8, 2, 1, 2, 8, yMax]

    matOut = np.full((matIn.shape[0], outDims), yMax)
    matActvGradInv = np.full((matIn.shape[0], outDims), slopesInv[0])

    for i in range(len(matIn)):
      for j in range(len(matIn[i].squeeze())):
        x = matIn[i].squeeze()[j] // divisor
        if x < joints[0]:
          matOut[i][j] = x // 4 + 88
          matActvGradInv[i][j] = slopesInv[1]
        if x < joints[1]:
          matOut[i][j] = x + 32
          matActvGradInv[i][j] = slopesInv[2]
        if x < joints[2]:
          matOut[i][j] = x * 2
          matActvGradInv[i][j] = slopesInv[3]
        if x < joints[3]:
          matOut[i][j] = x - 32
          matActvGradInv[i][j] = slopesInv[4]
        if x < joints[4]:
          matOut[i][j] = x // 4 - 88
          matActvGradInv[i][j] = slopesInv[5]
        if x < joints[5]:
          matOut[i][j] = yMin
          matActvGradInv[i][j] = slopesInv[6]
    return matOut.astype(int), matActvGradInv

In [129]:
def scalarL2Loss(y, yHat):
    return (yHat - y) * (yHat - y) // 2

def scalarL2LossDelta(y, yHat):
    return (yHat - y)

def batchL2Loss(yMat, yHatMat):
    # IMPORTANT: One sumLoss value per one sample
    accumLoss = 0
    # Each row corresponds to one input
    for i in range(len(yMat)):
      columnLoss = 0
      for j in range(len(yMat[i])):
        columnLoss += scalarL2Loss(yMat[i][j], yHatMat[i][j])
      accumLoss += columnLoss
    return accumLoss

def batchL2LossDelta(yMat, yHatMat):
    # Assumption: 1 input -> 1 scalar sumLoss value
    # for 1 output of dimention T, lossDeltaMat = (1, T)
    lossDeltaMat = np.zeros((yMat.shape[0], yMat.shape[1]))
    accumLossDelta = 0
    # Per each input item
    for i in range(len(yMat)):
      columnLossDelta = 0
      for j in range(len(yMat[i])):
        scalarLossDelta = scalarL2LossDelta(yMat[i][j], yHatMat[i].squeeze()[j])
        lossDeltaMat[i][j] = scalarLossDelta
        columnLossDelta += scalarLossDelta
      accumLossDelta += columnLossDelta

    # return sum! (average is meaningless)
    return lossDeltaMat.astype(int), accumLossDelta

In [130]:
class FCLayer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size, outLayer = False, debug=False):
      self.input_size = input_size
      self.output_size = output_size
      self.outLayer = outLayer
      self.debug = debug
      self.weights = np.zeros((input_size, output_size)).astype(int)
      self.bias = np.zeros((1, output_size)).astype(int)
      self.mDfaWeight = np.zeros((1, 1)).astype(int)

    # returns output for a given input
    def forward(self, input_data):
        self.input = input_data
        dot = self.input @ self.weights
        dot += self.bias
        output, self.matActvGradInv = pocketTanh(dot, 8, self.input_size, self.output_size)
        return output

    def backward(self, lastLayerDeltasMat, lrInv):   
      mDeltas = self.computeDeltas(lastLayerDeltasMat, lrInv)
      batchSize = len(mDeltas) # 1 for one item    
      mWeightUpdate = self.input.T @ mDeltas
      mWeightUpdate = (mWeightUpdate // lrInv).astype(int)
      self.weights -= mWeightUpdate

      ones = np.ones((batchSize, 1)).astype(int)
      mBiasUpdate = mDeltas.T @ ones
      mBiasUpdate = (mBiasUpdate.T // lrInv).astype(int)
      self.bias -= mBiasUpdate

      return lastLayerDeltasMat

    def computeDeltas(self, lastLayerDeltasMat, lrInv):
      if self.outLayer:
        mDeltas = np.floor_divide(lastLayerDeltasMat, self.matActvGradInv)
      else:
        dot = lastLayerDeltasMat @ self.mDfaWeight
        mDeltas = np.floor_divide(dot, self.matActvGradInv)
      return mDeltas

In [131]:
class MaxPoolLayer:
  
  def __init__(self, kernel_size, stride=(1, 1)):
    self.kernel_size = kernel_size
    self.stride = stride

  def forward(self, batch, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
    return np.array([_max(image, self.kernel_size, self.stride) for image in batch])

  def backward(self, lastLayerDeltasMat, lrInv, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
      return lastLayerDeltasMat

def _max(image, kernel_size, stride):
    x_s = stride[1]
    y_s = stride[0]

    x_k = kernel_size[1]
    y_k = kernel_size[0]

    # print(image)
    x_d = len(image[0])
    y_d = len(image)

    x_o = ((x_d - x_k) // x_s) + 1
    y_o = ((y_d - y_k) // y_s) + 1

    def get_submatrix(matrix, x, y):
        index_row = y * y_s
        index_column = x * x_s
        return matrix[index_row: index_row + y_k, index_column: index_column + x_k]

    return [[np.max(get_submatrix(image, x, y).flatten()) for x in range(0, x_o)] for y in range(0, y_o)]

In [132]:
class FlattenLayer:

    def __init__(self):
        pass

    def forward(self, image):
        return image.reshape(image.shape[0], image.shape[1]*image.shape[2])

    def backward(self, lastLayerDeltasMat, lrInv):
      return lastLayerDeltasMat

In [133]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # test
    def test(self, x_test, y_test):
      # sample dimension first
      samples = len(x_test)
      corr = 0
      for j in range(samples):
          # forward propagation
          pred = self.predict(x_test[j])

          if pred == y_test[j]:
            corr += 1
      return corr / samples * 100

    # predict output for given input
    def predict(self, input_data):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output)

        return output.argmax()

    # train the network
    def fit(self, x_train, y_train, epochs, miniBatchSize, lrInv):
        # sample dimension first
        samples = len(x_train)
        train_accs, val_accs = [], []

        # training loop
        for i in range(epochs):
            sumLoss = 0
            sumLossDelta = 0
            epochNumCorrect = 0
            numIter = int(samples/miniBatchSize)

            for j in range(numIter):
                batchNumCorrect = 0
                idxStart = j * miniBatchSize
                idxEnd = idxStart + miniBatchSize

                miniBatchImages = x_train[idxStart:idxEnd]
                miniBarchTargets = y_train[idxStart:idxEnd]

                # forward propagation
                output = miniBatchImages
                
                for layer in self.layers:
                  output = layer.forward(output)

                sumLoss += batchL2Loss(miniBarchTargets, output)
                lossDeltaMat, sumLossDelta = batchL2LossDelta(miniBarchTargets, output)

                for r in range(miniBatchSize):
                    if miniBarchTargets[r].argmax() == output[r].argmax():
                        batchNumCorrect += 1
                
                for layer in reversed(self.layers):
                    layer.backward(lossDeltaMat, lrInv)
                
                epochNumCorrect += batchNumCorrect;

            acc = epochNumCorrect/samples * 100
            # print("Epoch: " + repr(i))
            # print("Train Accuracy: " + repr(acc) + " %")
            train_accs.append(acc)
            val_accs.append(self.test(x_val, y_val))
        return train_accs, val_accs

## Experiments

### Training

In [134]:
# Params
samples = 1000
epochs = 150
lrs = [128, 256, 512, 1024, 2048, 4096]
bs = 25

In [135]:
with open("accsBS25.pkl", "rb") as f:
    DFAWeights = pickle.load(f)
    old = pickle.load(f)

In [136]:
DFAWeightsUniform = DFAWeights['Uniform']
DFAWeightsNormal = DFAWeights['Normal']

In [137]:
def train(DFAWeights, j, lr):
  # Net structure
  net = Network()
  net.add(FCLayer(4, 2))
  net.add(FCLayer(2, 3, outLayer=True))

  net.layers[0].mDfaWeight = DFAWeights[j]
  # Train
  _, val_acc = net.fit(x_train, y_train, epochs=epochs, miniBatchSize=bs, lrInv=lr)
  
  return {str(lr)+'-'+str(j): [val_acc, [net.layers[0].weights, net.layers[0].bias, net.layers[1].weights, net.layers[1].bias]]}

In [138]:
start_time = time.time()
resU = Parallel(n_jobs=-1)(delayed(train)(DFAWeightsUniform, j, lr) for lr in lrs for j in range(samples))
end_time = time.time()
print("Total time: " + str(end_time-start_time) + " s")

Total time: 1923.3325254917145 s


In [139]:
accsLR_U = {}
for k in range(len(lrs)):
    accsMean = []
    accsMax = []
    W = []
    for j in range(samples):
        max = np.max(resU[samples*k+j][str(lrs[k])+'-'+str(j)][0])
        mean = np.mean(resU[samples*k+j][str(lrs[k])+'-'+str(j)][0][100:])
        weights = resU[samples*k+j][str(lrs[k])+'-'+str(j)][1]
        accsMean.append(mean/100)
        accsMax.append(max/100)
        W.append(weights)
    accsLR_U[lrs[k]] = [[accsMean, accsMax], W]

In [140]:
start_time = time.time()
resN = Parallel(n_jobs=-1)(delayed(train)(DFAWeightsNormal, j, lr) for lr in lrs for j in range(samples))
end_time = time.time()
print("Total time: " + str(end_time-start_time) + " s")

Total time: 1945.954912185669 s


In [141]:
accsLR_N = {}
for k in range(len(lrs)):
    accsMean = []
    accsMax = []
    W = []
    for j in range(samples):
        max = np.max(resN[samples*k+j][str(lrs[k])+'-'+str(j)][0])
        mean = np.mean(resN[samples*k+j][str(lrs[k])+'-'+str(j)][0][100:])
        weights = resN[samples*k+j][str(lrs[k])+'-'+str(j)][1]
        accsMean.append(mean/100)
        accsMax.append(max/100)
        W.append(weights)
    accsLR_N[lrs[k]] = [[accsMean, accsMax], W]

In [142]:
accsLR = {'Uniform': accsLR_U, 'Normal': accsLR_N}
DFAWeights = {'Uniform': DFAWeightsUniform, 'Normal': DFAWeightsNormal}

In [143]:
with open("accsBS25.pkl", "wb") as f:
    pickle.dump(DFAWeights, f)
    pickle.dump(accsLR, f)